In [1181]:
import cv2
import numpy as np
import pytesseract


In [1182]:
image = cv2.imread('images/receipt_image.jpg', 0)
color = cv2.imread('images/receipt_image.jpg')
hsv = cv2.cvtColor(color, cv2.COLOR_BGR2HSV)

hue, saturation, value = cv2.split(hsv)

In [1183]:
def show(image):
    
    resized_height = 720
    percent = resized_height / len(image)
    resized_width = int(percent * len(image[0]))
    gray = cv2.resize(image,(resized_width,resized_height))

    cv2.imshow('cringe', gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [1184]:
thresh = cv2.threshold(hue, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
show(thresh)

In [1185]:
eroded = cv2.erode(thresh, kernel=None, iterations=7)
dilated = cv2.dilate(eroded, kernel=None, iterations=15)
eroded = cv2.erode(dilated, kernel=None, iterations=100)
dilated = cv2.dilate(eroded, kernel=None, iterations=100)
show(dilated)

In [1186]:
def extract_largest_rectangular_contour(color_image, preprocessed_image):
    contours, _ = cv2.findContours(preprocessed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = None
    largest_area = 0
    largest_approx = None
    
    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        if len(approx) == 4:
            area = cv2.contourArea(contour)
            if area > largest_area:
                largest_area = area
                largest_contour = contour
                largest_approx = approx
                
    if largest_contour is not None:
        cv2.drawContours(color_image, [largest_approx], -1, (0, 255, 0), 3)
        # x, y, w, h = cv2.boundingRect(largest_contour)
        # cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        return color_image, largest_approx
    
    return color_image, None

In [1187]:
im, cor = extract_largest_rectangular_contour(color, dilated)
show(im)

In [1188]:
cor=cor.reshape((4,2))

top = np.min(cor[:, 1])-1
bottom = np.max(cor[:, 1])+1
left = np.min(cor[:, 0])-1
right = np.max(cor[:, 0])+1

cropped = image[top:bottom, left:right]
show(cropped)

In [1189]:
def order_points(coords):
    pts = coords - [left, top]
    pts = pts.reshape(4, 2)
    sum_pts = pts.sum(axis=1)
    diff_pts = np.diff(pts, axis=1)
    
    top_left = pts[np.argmin(sum_pts)]
    top_right = pts[np.argmin(diff_pts)]
    bottom_right = pts[np.argmax(sum_pts)]
    bottom_left = pts[np.argmax(diff_pts)]
    
    return np.array([top_left, top_right, bottom_right, bottom_left])

In [1190]:
def calculate_size(img, ordered):
    width = int(img.shape[1] * 0.8)
    aspect_ratio = np.linalg.norm(ordered[0] - ordered[3]) / np.linalg.norm(ordered[0] - ordered[1])
    return width, int(width * aspect_ratio)

In [1191]:
def process_image(img, contour):
    ordered = order_points(contour)
    img_with_pts = img.copy()
    for pt in ordered:
        img_with_pts = cv2.circle(img_with_pts, tuple(pt), 10, (255, 255, 255), -1)
  
    w, h = calculate_size(img, ordered)
    
    pts1 = np.float32(ordered)
    pts2 = np.float32([[0, 0], [w, 0], [w, h], [0, h]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    
    corrected_img = cv2.warpPerspective(img, matrix, (w, h))
    
    return img_with_pts, corrected_img

In [1192]:
img_with_pts, corrected_img = process_image(cropped, cor)
show(corrected_img)

In [1193]:
# thresh_cropped = cv2.adaptiveThreshold(corrected_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
# show(thresh_cropped)

In [1194]:
# med_blur = cv2.medianBlur(thresh_cropped, 3)
# show(med_blur)

In [1195]:
blur = cv2.GaussianBlur(corrected_img, (5, 5), 0)
mask=cv2.subtract(corrected_img,blur)
sharpened = cv2.addWeighted(corrected_img, 1, mask,0.5, 0)
# show(sharpened)

In [1196]:
equalized_img = cv2.equalizeHist(corrected_img)
# show(equalized_img)

In [1197]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
clahe_img = clahe.apply(corrected_img)
# show(clahe_img)

In [1198]:
text = pytesseract.image_to_string(clahe_img)
sentences=text.split('\n')
for s in sentences:
    print(s)

FINGER CHIPS 1
CHL GHEE RAST BONLESS
FLL

CHI GHEE ROAST BONLESS
HALE

CRISPY CORN
DRAGON CHT
ANIAL RAHA FRY.
ANGARA KABAB

Food Total

TEACHERS 60 HL PET
YO LUXURY 30K.

BLACK 005 30ML
BUDWISER MAGNUM 330ML
ROMONOV. VODKA 30HL
ADVAN CHIPS

WHISKY SOUR

 

